In [ ]:
from easifish_pipeline import easifish_registration_pipeline
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline
from bigstream.piecewise_transform import distributed_apply_transform
from ClusterWrap.clusters import local_cluster, janelia_lsf_cluster
from ClusterWrap.decorator import cluster as cluster_constructor

import zarr
import numpy as np
import os

##### Registration of data functional anatomy volume to confocal microscopy volume

In [ ]:
# Load sample data with 4 blocks

sample_num = 0 # test data sample
prefix  = 'path/to/ptoject/data'

# anatomy to confocal
write_directory =os.path.join(prefix , f'/registration/test_samples/sample{sample_num}/anatomy2confocal/')

prefix_src = os.path.join( prefix , f'/src/test_samples/sample{sample_num}')
reference = zarr.open(os.path.join(prefix_src, 'confocal.zarr'), 'r')
modified = zarr.open(os.path.join(prefix_src, 'anatomy.zarr'), 'r')

# load reference volume
highres_level = 0
lowres_level = 2

fix_lowres = reference[f's{lowres_level}'][:]
fix_highres = reference[f's{highres_level}']  # lazy load

# load volume that will be registered
mov_lowres = modified[f's{lowres_level}'][:]
mov_highres = modified[f's{highres_level}']

# Define voxel spacings (in micrometers, for example)
fix_lowres_spacing = np.array(reference.attrs['multiscales'][0]['datasets'][lowres_level]['coordinateTransformations'][0]['scale'])
fix_highres_spacing = np.array(reference.attrs['multiscales'][0]['datasets'][highres_level]['coordinateTransformations'][0]['scale'])

mov_lowres_spacing = np.array(modified.attrs['multiscales'][0]['datasets'][lowres_level]['coordinateTransformations'][0]['scale'])
mov_highres_spacing = np.array(modified.attrs['multiscales'][0]['datasets'][highres_level]['coordinateTransformations'][0]['scale'])

# Threshold values > 65000 and convert to uint16 (values are lower than 65k anyway)
fix_lowres = np.round(np.clip(fix_lowres, 0, 65000)).astype(np.uint16)
mov_lowres = np.clip(mov_lowres, 0, 65000).astype(np.uint16)
